In [30]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Load the saved model
model_path = r'C:\Users\RDITLADI\Documents\AMPL\code\all_run_dir\drag_run_dir\results_drag_coefficient\saved_models\Cd_drag_estimate_top_0_model.keras'
base_model = tf.keras.models.load_model(model_path)

In [31]:
# Load dataset
data_path = r'c:\Users\RDITLADI\Documents\AMPL\code\all_run_dir\drag_run_dir\drag_data\sc1095_30AUG2023_noZeros.drag.csv'
data = pd.read_csv(data_path)

In [32]:
# Separate features and target variable
X = data[['Temp', 'LWC', 'Mach_env', 'AoA_env', 'Mach_c81', 'AoA_c81']]
y = data['Cd']

In [34]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [35]:
# Convert to numpy arrays for model compatibility
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [36]:
# Number of models for bagging
n_bagging_models = 10

# Store models
models = []

In [37]:
for i in range(n_bagging_models):
    # Resample the training data
    X_train_resampled, y_train_resampled = resample(X_train, y_train, n_samples=len(y_train), random_state=i, replace=True)


In [39]:
# Train the model on the resampled data
model = tf.keras.models.clone_model(base_model)
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_resampled, y_train_resampled, epochs=50, verbose=0)

In [40]:
 # Store the trained model
models.append(model)

In [41]:
# Ensemble predictions on test data
ensemble_pred = np.zeros(len(y_test))
for model in models:
    ensemble_pred += model.predict(X_test).flatten()

408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [42]:
ensemble_pred /= n_bagging_models

In [43]:
# Evaluate the ensemble
mse = mean_squared_error(y_test, ensemble_pred)
print(f'Ensemble MSE: {mse}')

Ensemble MSE: 0.07340628325899827
